# Download Torrents Directly to Google Drive

This notebook allows you to download a file from a magnet link directly to your Google Drive using Google Colab's servers.

### Instructions:
1.  **Paste your magnet link** into the `magnet_link` field below.
2.  (Optional) Change the `save_path` if you want to save the files to a different folder in your Google Drive.
3.  Click the **Play button** (▶️) to run the cell.
4.  Follow the link to **authorize Google Drive** access and paste the code back.
5.  Wait for the download to complete.

In [ ]:
#@title Download Torrent to Google Drive
#@markdown --- 
#@markdown ### 1. Connect to Google Drive
#@markdown This will mount your Google Drive to the Colab environment.
from google.colab import drive
drive.mount('/content/drive')

#@markdown ### 2. Install Required Library
#@markdown This installs the `libtorrent` library for handling torrents.
!pip install -q libtorrent

#@markdown ---
#@markdown ### 3. Start Download
#@markdown Paste your magnet link below and run the cell.
magnet_link = "" #@param {type:"string"}
#@markdown The path in your Google Drive where files will be saved:
save_path = "/content/drive/MyDrive/TorrentDownloads/" #@param {type:"string"}

import libtorrent as lt
import time
import os

# Create the save directory if it doesn't exist
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Start the torrent session
ses = lt.session({'listen_interfaces': '0.0.0.0:6881'})

# Parse the magnet link and set the save path
params = lt.parse_magnet_uri(magnet_link)
params.save_path = save_path
h = ses.add_torrent(params)

print("Fetching torrent metadata...")
while not h.has_metadata():
    time.sleep(1)

print(f"Starting download for: {h.status().name}")

# List of possible torrent states
state_str = [
    'queued', 'checking', 'downloading metadata', 'downloading',
    'finished', 'seeding', 'allocating', 'checking resume data'
]

try:
    while h.status().state != lt.torrent_status.seeding:
        s = h.status()
        
        # Safely get the current state string
        current_state = "unknown"
        if s.state < len(state_str):
            current_state = state_str[s.state]
        
        print('\r%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s' % \
                (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
                s.num_peers, current_state), end=' ')
        
        time.sleep(1)

        # Break the loop if download is finished to avoid endless seeding
        if s.state == lt.torrent_status.finished:
            print("\nDownload finished. Entering seeding mode briefly...")
            break

except KeyboardInterrupt:
    print("\nOperation stopped by user.")

print(f"\n\nDownload Complete!")
print(f"Files are saved in your Google Drive at:")
print(save_path)